In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
import chime
%load_ext chime

Loading train, validation and test sets

In [3]:
ds = tfds.load('malaria', 
               split=('train[:60%]', 'train[60%:80%]', 'train[80%:]'), 
               shuffle_files=False,
              data_dir='./data',
               batch_size=8,
              download=False,
              as_supervised=True,
              with_info=False)
ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

# Model optimization

In [4]:
inputs = keras.Input(shape=(None, None, 3))
x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
kernel_size = 2
x = keras.layers.Conv2D(
    filters=256,
    kernel_size=kernel_size,
    strides=(1, 1),
    padding="valid",
    activation='relu')(x)
x = keras.layers.MaxPooling2D(
    pool_size=(2, 2), 
    strides=None, 
    padding="valid")(x)
x = keras.layers.Conv2D(
    filters=512,
    kernel_size=kernel_size,
    strides=(1, 1),
    padding="valid",
    activation='relu')(x)
x = keras.layers.GlobalMaxPooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=512,
                        activation="relu",
                      kernel_regularizer=None)(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='Adam',
      loss='binary_crossentropy',
      metrics=['accuracy'])

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 rescaling (Rescaling)       (None, None, None, 3)     0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 256)   3328      
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 256)  0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 512)   524800    
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                              

In [6]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=5,
        verbose=1,
        mode="max",
        baseline=None,
        restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(
        filepath='/content/gdrive/MyDrive/datasets/Malaria/checkpoints/model_{epoch}',
        save_freq='epoch')
    ]

In [7]:
%%time
%%chime
history = model.fit(
    ds[0],
    epochs=50,
    verbose="auto",
    callbacks=callbacks,
    validation_data=ds[1]
)

Epoch 1/50
 155/2067 [=>............................] - ETA: 59:44 - loss: 0.5024 - accuracy: 0.7073

KeyboardInterrupt: 

In [8]:
pd.DataFrame({'accuracy': history.history['accuracy'],
              'val_accuracy': history.history['val_accuracy']}).plot.line()

NameError: name 'history' is not defined

In [ ]:
model.evaluate(ds[2])